In [1]:
"""
Main Insight Generator
Supports both pure LLM and evidence-based generation via OpenRouter API.
"""
import os
import json
import time
from typing import Dict, Any, List, Optional
import requests
from prompt_templates import PromptTemplates, RegionSpecificPrompts
from pubmed_integration import EvidenceRetriever


class OpenRouterClient:
    """Client for OpenRouter API."""
    
    def __init__(self, api_key: Optional[str] = None):
        """
        Initialize OpenRouter client.
        
        Args:
            api_key: OpenRouter API key (or set OPENROUTER_API_KEY env var)
        """
        self.api_key = api_key or os.getenv('OPENROUTER_API_KEY')
        if not self.api_key:
            raise ValueError("OpenRouter API key required. Set OPENROUTER_API_KEY environment variable.")
        
        self.base_url = "https://openrouter.ai/api/v1/chat/completions"
        self.default_model = "x-ai/grok-4.1-fast"  # Can be changed
    
    def generate(self, prompt: str, model: Optional[str] = None, 
                temperature: float = 0.7, max_tokens: int = 2000) -> str:
        """
        Generate completion via OpenRouter.
        
        Args:
            prompt: Input prompt
            model: Model to use (default: claude-3.5-sonnet)
            temperature: Sampling temperature
            max_tokens: Maximum tokens to generate
            
        Returns:
            Generated text response
        """
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://dyk-health-insights.com",  # Optional but recommended
        }
        
        data = {
            "model": model or self.default_model,
            "messages": [
                {"role": "user", "content": prompt}
            ],
            "temperature": temperature,
            "max_tokens": max_tokens
        }
        
        try:
            response = requests.post(self.base_url, headers=headers, json=data, timeout=60)
            response.raise_for_status()
            
            result = response.json()
            return result['choices'][0]['message']['content']
        
        except requests.exceptions.RequestException as e:
            print(f"OpenRouter API error: {e}")
            if hasattr(e, 'response') and e.response is not None:
                print(f"Response: {e.response.text}")
            raise


class InsightGenerator:
    """Main insight generation orchestrator."""
    
    def __init__(self, api_key: Optional[str] = None, 
                 pubmed_email: Optional[str] = None,
                 pubmed_api_key: Optional[str] = None,
                 model: str = "anthropic/claude-3.5-sonnet"):
        """
        Initialize insight generator.
        
        Args:
            api_key: OpenRouter API key
            pubmed_email: Email for PubMed API
            pubmed_api_key: Optional PubMed API key for higher rate limits
            model: LLM model to use
        """
        self.llm = OpenRouterClient(api_key)
        self.evidence_retriever = EvidenceRetriever(pubmed_email, pubmed_api_key)
        self.model = model
        self.templates = PromptTemplates()
    
    def generate_pure_llm(self, cohort_spec: Dict[str, Any], 
                         template_type: str = "risk_amplification",
                         region: str = "singapore") -> Dict[str, Any]:
        """
        Generate insight using pure LLM knowledge (no external tools).
        
        Args:
            cohort_spec: Cohort specification with params and description
            template_type: Type of insight template
            region: Target region
            
        Returns:
            Generated insight dictionary
        """
        cohort_params = cohort_spec['cohort_params']
        cohort_description = cohort_spec['description']
        
        # Generate prompt
        prompt = self.templates.pure_llm_insight_generation(
            cohort_description=cohort_description,
            cohort_params=cohort_params,
            region=region,
            template_type=template_type
        )
        
        # Add region-specific context
        if region == "singapore":
            prompt += RegionSpecificPrompts.singapore_context()
        else:
            prompt += RegionSpecificPrompts.global_context()
        
        # Generate
        try:
            response = self.llm.generate(prompt, model=self.model, temperature=0.7)
            
            # Parse JSON response
            insight = self._parse_json_response(response)
            
            # Add metadata
            insight['cohort_id'] = cohort_spec['cohort_id']
            insight['cohort_params'] = cohort_params
            insight['generation_method'] = 'pure_llm'
            insight['model_used'] = self.model
            insight['template_type'] = template_type
            insight['region'] = region
            
            return insight
        
        except Exception as e:
            print(f"Error generating pure LLM insight: {e}")
            return None
    
    def generate_evidence_based(self, cohort_spec: Dict[str, Any],
                               template_type: str = "risk_amplification",
                               region: str = "singapore",
                               max_sources: int = 5) -> Dict[str, Any]:
        """
        Generate insight using external evidence (PubMed).
        
        Args:
            cohort_spec: Cohort specification
            template_type: Type of insight template
            region: Target region
            max_sources: Maximum evidence sources to retrieve
            
        Returns:
            Generated insight dictionary with evidence
        """
        cohort_params = cohort_spec['cohort_params']
        cohort_description = cohort_spec['description']
        
        # Retrieve evidence
        print(f"Retrieving evidence for: {cohort_description}")
        evidence_data = self.evidence_retriever.retrieve_for_cohort(
            cohort_params=cohort_params,
            max_sources=max_sources
        )
        
        if not evidence_data['articles']:
            print("No evidence found, falling back to pure LLM")
            return self.generate_pure_llm(cohort_spec, template_type, region)
        
        print(f"Retrieved {evidence_data['total_sources']} sources")
        
        # Generate prompt with evidence
        prompt = self.templates.evidence_based_insight_generation(
            cohort_description=cohort_description,
            cohort_params=cohort_params,
            evidence_context=evidence_data['evidence_context'],
            region=region,
            template_type=template_type
        )
        
        # Add region-specific context
        if region == "singapore":
            prompt += RegionSpecificPrompts.singapore_context()
        
        # Generate
        try:
            response = self.llm.generate(prompt, model=self.model, temperature=0.6)
            
            # Parse JSON response
            insight = self._parse_json_response(response)
            
            # Add metadata
            insight['cohort_id'] = cohort_spec['cohort_id']
            insight['cohort_params'] = cohort_params
            insight['generation_method'] = 'evidence_based'
            insight['model_used'] = self.model
            insight['template_type'] = template_type
            insight['region'] = region
            insight['evidence_sources'] = evidence_data['articles']
            insight['search_queries'] = evidence_data['queries']
            
            return insight
        
        except Exception as e:
            print(f"Error generating evidence-based insight: {e}")
            return None
    
    def validate_insight(self, insight: Dict[str, Any]) -> Dict[str, Any]:
        """
        Validate insight using secondary LLM call.
        
        Args:
            insight: Generated insight to validate
            
        Returns:
            Validation results
        """
        prompt = self.templates.validation_prompt(
            insight=insight,
            cohort_params=insight.get('cohort_params', {})
        )
        
        try:
            response = self.llm.generate(prompt, model=self.model, temperature=0.3)
            validation = self._parse_json_response(response)
            return validation
        
        except Exception as e:
            print(f"Error validating insight: {e}")
            return None
    
    def batch_generate(self, cohort_specs: List[Dict[str, Any]],
                      method: str = "pure_llm",
                      insights_per_cohort: int = 3,
                      template_types: Optional[List[str]] = None,
                      region: str = "singapore",
                      validate: bool = True,
                      rate_limit_delay: float = 1.0) -> List[Dict[str, Any]]:
        """
        Generate insights for multiple cohorts in batch.
        
        Args:
            cohort_specs: List of cohort specifications
            method: "pure_llm" or "evidence_based"
            insights_per_cohort: Number of insights to generate per cohort
            template_types: List of template types to cycle through
            region: Target region
            validate: Whether to validate each insight
            rate_limit_delay: Delay between API calls (seconds)
            
        Returns:
            List of all generated insights
        """
        if template_types is None:
            template_types = ["risk_amplification", "protective_factors", 
                            "behavior_change", "early_detection"]
        
        all_insights = []
        total_cohorts = len(cohort_specs)
        
        for idx, cohort_spec in enumerate(cohort_specs, 1):
            print(f"\n{'='*80}")
            print(f"Cohort {idx}/{total_cohorts}: {cohort_spec['description']}")
            print(f"{'='*80}")
            
            cohort_insights = []
            
            for i in range(insights_per_cohort):
                template_type = template_types[i % len(template_types)]
                
                print(f"  Generating insight {i+1}/{insights_per_cohort} ({template_type})...")
                
                # Generate insight
                if method == "pure_llm":
                    insight = self.generate_pure_llm(
                        cohort_spec=cohort_spec,
                        template_type=template_type,
                        region=region
                    )
                elif method == "evidence_based":
                    insight = self.generate_evidence_based(
                        cohort_spec=cohort_spec,
                        template_type=template_type,
                        region=region
                    )
                else:
                    raise ValueError(f"Unknown method: {method}")
                
                if insight:
                    # Validate if requested
                    if validate:
                        print("  Validating...")
                        validation = self.validate_insight(insight)
                        if validation:
                            insight['validation'] = validation
                            print(f"  Validation score: {validation.get('overall_score', 'N/A')}/100")
                    
                    cohort_insights.append(insight)
                    all_insights.append(insight)
                    print("  ✓ Success")
                else:
                    print("  ✗ Failed")
                
                # Rate limiting
                time.sleep(rate_limit_delay)
            
            print(f"Completed {len(cohort_insights)}/{insights_per_cohort} insights for this cohort")
        
        print(f"\n{'='*80}")
        print(f"BATCH COMPLETE: Generated {len(all_insights)} total insights")
        print(f"{'='*80}")
        
        return all_insights
    
    def _parse_json_response(self, response: str) -> Dict[str, Any]:
        """Parse JSON from LLM response, handling markdown code blocks."""
        # Remove markdown code blocks if present
        response = response.strip()
        if response.startswith("```json"):
            response = response[7:]
        if response.startswith("```"):
            response = response[3:]
        if response.endswith("```"):
            response = response[:-3]
        
        response = response.strip()
        
        try:
            return json.loads(response)
        except json.JSONDecodeError as e:
            print(f"JSON parse error: {e}")
            print(f"Response: {response[:500]}")
            raise


def save_insights(insights: List[Dict[str, Any]], output_path: str):
    """Save generated insights to JSON file."""
    # Add timestamp
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
    
    output = {
        'generated_at': timestamp,
        'total_insights': len(insights),
        'insights': insights
    }
    
    with open(output_path, 'w') as f:
        json.dump(output, f, indent=2)
    
    print(f"\nSaved {len(insights)} insights to {output_path}")


In [2]:
PUBMED_API_KEY = os.getenv("PUBMED_API_KEY")
PUBMED_EMAIL = os.getenv("PUBMED_EMAIL")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

In [4]:
# create generator
generator = InsightGenerator(api_key=OPENROUTER_API_KEY,
                             pubmed_email=PUBMED_EMAIL,
                             pubmed_api_key=PUBMED_API_KEY,
                             model="x-ai/grok-4.1-fast")

In [6]:
# define priority_cohorts
priority_cohorts = [
    {
      "cohort_id": "cohort_0025",
      "cohort_params": {
        "age_group": "18-29",
        "gender": "male"
      },
      "min_insights": 3,
      "priority_level": 2,
      "description": "18-29 years old, male"
    },
    {
      "cohort_id": "cohort_0026",
      "cohort_params": {
        "age_group": "18-29",
        "gender": "female"
      },
      "min_insights": 3,
      "priority_level": 2,
      "description": "18-29 years old, female"
    }
]

In [ ]:
all_insights = generator.batch_generate(priority_cohorts,
                                        method="pure_llm",
                                        insights_per_cohort=3,
                                        template_types=None,
                                        region="singapore",
                                        validate=True,
                                        rate_limit_delay=1.0)


Cohort 1/2: 18-29 years old, male
  Generating insight 1/3 (risk_amplification)...
  Validating...
  Validation score: 95/100
  ✓ Success
  Generating insight 2/3 (protective_factors)...
  Validating...
  Validation score: 98/100
  ✓ Success
  Generating insight 3/3 (behavior_change)...
  Validating...
  Validation score: 98/100
  ✓ Success
Completed 3/3 insights for this cohort

Cohort 2/2: 18-29 years old, female
  Generating insight 1/3 (risk_amplification)...
  Validating...
  Validation score: 95/100
  ✓ Success
  Generating insight 2/3 (protective_factors)...
  Validating...
  Validation score: 85/100
  ✓ Success
  Generating insight 3/3 (behavior_change)...
  Validating...
  Validation score: 100/100
  ✓ Success
Completed 3/3 insights for this cohort

BATCH COMPLETE: Generated 6 total insights


In [ ]:
all_insights

[{'hook': 'Did you know Singaporean men aged 18-29 have nearly 3x higher suicide rates than women their age?',
  'explanation': 'Young men in Singapore face high stress from National Service, academics, and early careers, compounded by stigma against seeking help. This elevates suicide risk significantly for this group, making early awareness and support crucial to prevent tragedy.',
  'action': 'If feeling stressed or low, call Samaritans of Singapore at 1800-221-4444 for free, confidential 24/7 support—no judgement.',
  'source_name': 'Samaritans of Singapore',
  'source_url': 'https://www.sos.org.sg/suicide-statistics',
  'health_domain': 'mental-health',
  'confidence': 'high',
  'numeric_claim': 'nearly 3x higher suicide rates',
  'cohort_id': 'cohort_0025',
  'cohort_params': {'age_group': '18-29', 'gender': 'male'},
  'generation_method': 'pure_llm',
  'model_used': 'x-ai/grok-4.1-fast',
  'template_type': 'risk_amplification',
  'region': 'singapore',
  'validation': {'overall_

In [58]:
validation = generator.validate_insight(insight)

In [59]:
validation

{'overall_score': 100,
 'factual_accuracy': {'score': 100, 'issues': []},
 'cohort_relevance': {'score': 100, 'issues': []},
 'source_faithfulness': {'score': 100, 'issues': []},
 'safety': {'score': 100, 'issues': []},
 'actionability': {'score': 100, 'issues': []},
 'recommendation': 'approve',
 'revision_suggestions': []}

In [60]:
validation = generator.validate_insight(insight)

In [61]:
validation

{'overall_score': 98,
 'factual_accuracy': {'score': 98,
  'issues': ["Hook phrasing 'smoking in your 40s nearly doubles' slightly interpretive; study shows current frequent smoking associated with ~73-95% increased odds in adults/males, including 40-49 age group, but not specifically causation from smoking starting/in 40s. Still medically accurate and plausible."]},
 'cohort_relevance': {'score': 100, 'issues': []},
 'source_faithfulness': {'score': 100, 'issues': []},
 'safety': {'score': 100, 'issues': []},
 'actionability': {'score': 100, 'issues': []},
 'recommendation': 'approve',
 'revision_suggestions': []}

In [ ]:
    {
      "cohort_id": "cohort_10019",
      "cohort_params": {
        "chronic_conditions": "hypertension"
      },
      "min_insights": 5,
      "priority_level": 1,
      "description": "hypertension"
    },
    {
      "cohort_id": "cohort_10020",
      "cohort_params": {
        "chronic_conditions": "high-cholesterol"
      },
      "min_insights": 5,
      "priority_level": 2,
      "description": "high cholesterol"
    },
    {

In [ ]:
"""
Validation Layer
Ensures insights conform to schema, have valid sources, and meet quality standards.
"""
import re
import json
from typing import Dict, Any, List, Tuple, Optional
from urllib.parse import urlparse
import requests
from difflib import SequenceMatcher



In [ ]:
class InsightValidator:
    """Validator for DYK insights."""
    
    # Whitelisted domains for sources
    WHITELISTED_DOMAINS = {
        'who.int', 'cdc.gov', 'nih.gov', 'pubmed.ncbi.nlm.nih.gov',
        'thelancet.com', 'nejm.org', 'bmj.com', 'jamanetwork.com',
        'healthhub.sg', 'moh.gov.sg', 'hpb.gov.sg'
    }
    
    def __init__(self, config: Optional[Dict] = None):
        """
        Initialize validator.
        
        Args:
            config: Optional configuration with custom rules
        """
        self.config = config or {}
        self.validation_results = []
    
    def validate_insight(self, insight: Dict[str, Any]) -> Dict[str, Any]:
        """
        Comprehensive validation of a single insight.
        
        Args:
            insight: Insight dictionary to validate
            
        Returns:
            Validation result with scores and issues
        """
        results = {
            'insight_id': insight.get('cohort_id', 'unknown'),
            'overall_valid': True,
            'issues': [],
            'warnings': [],
            'scores': {}
        }
        
        # 1. Schema conformity
        schema_result = self._validate_schema(insight)
        results['scores']['schema'] = schema_result['score']
        if schema_result['issues']:
            results['issues'].extend(schema_result['issues'])
            results['overall_valid'] = False
        
        # 2. Source verification
        source_result = self._validate_source(insight)
        results['scores']['source'] = source_result['score']
        if source_result['issues']:
            results['issues'].extend(source_result['issues'])
        if source_result['warnings']:
            results['warnings'].extend(source_result['warnings'])
        
        # 3. Numeric plausibility
        numeric_result = self._validate_numeric_claims(insight)
        results['scores']['numeric'] = numeric_result['score']
        if numeric_result['issues']:
            results['issues'].extend(numeric_result['issues'])
        
        # 4. Content quality
        quality_result = self._validate_content_quality(insight)
        results['scores']['quality'] = quality_result['score']
        if quality_result['issues']:
            results['issues'].extend(quality_result['issues'])
        if quality_result['warnings']:
            results['warnings'].extend(quality_result['warnings'])
        
        # Calculate overall score
        scores = list(results['scores'].values())
        results['overall_score'] = sum(scores) / len(scores) if scores else 0
        
        # Mark as invalid if overall score is too low
        if results['overall_score'] < 60:
            results['overall_valid'] = False
        
        return results
    
    def _validate_schema(self, insight: Dict[str, Any]) -> Dict[str, Any]:
        """Validate schema conformity."""
        issues = []
        score = 100
        
        # Required fields
        required_fields = ['hook', 'explanation', 'action', 'cohort_params']
        for field in required_fields:
            if field not in insight or not insight[field]:
                issues.append(f"Missing required field: {field}")
                score -= 25
        
        # Check field types
        if 'cohort_params' in insight and not isinstance(insight['cohort_params'], dict):
            issues.append("cohort_params must be a dictionary")
            score -= 10
        
        # Check field lengths
        if 'hook' in insight:
            hook_words = len(insight['hook'].split())
            if hook_words > 25:
                issues.append(f"Hook too long: {hook_words} words (max 25)")
                score -= 10
            if hook_words < 5:
                issues.append(f"Hook too short: {hook_words} words (min 5)")
                score -= 10
        
        if 'explanation' in insight:
            exp_words = len(insight['explanation'].split())
            if exp_words < 30 or exp_words > 80:
                issues.append(f"Explanation length suboptimal: {exp_words} words (target 40-60)")
                score -= 5
        
        if 'action' in insight:
            action_words = len(insight['action'].split())
            if action_words > 40:
                issues.append(f"Action too long: {action_words} words (max 40)")
                score -= 10
        
        return {'score': max(0, score), 'issues': issues}
    
    def _validate_source(self, insight: Dict[str, Any]) -> Dict[str, Any]:
        """Validate source credibility and accessibility."""
        issues = []
        warnings = []
        score = 100
        
        source_url = insight.get('source_url', '')
        source_name = insight.get('source_name', '')
        
        if not source_name:
            issues.append("Missing source name")
            score -= 30
        
        if not source_url or source_url == 'general medical knowledge':
            warnings.append("No specific source URL provided")
            score -= 10
        else:
            # Validate URL format
            try:
                parsed = urlparse(source_url)
                if not parsed.scheme or not parsed.netloc:
                    issues.append(f"Invalid URL format: {source_url}")
                    score -= 20
                else:
                    # Check domain whitelist
                    domain = parsed.netloc.lower()
                    # Remove www. prefix
                    domain = domain.replace('www.', '')
                    
                    if not any(domain.endswith(whitelist) for whitelist in self.WHITELISTED_DOMAINS):
                        warnings.append(f"Source domain not in whitelist: {domain}")
                        score -= 10
            
            except Exception as e:
                issues.append(f"Error parsing URL: {e}")
                score -= 20
        
        # Check source tier if available
        source_tier = insight.get('source_tier')
        if source_tier and source_tier > 3:
            warnings.append(f"Low-tier source (tier {source_tier})")
            score -= 5
        
        return {'score': max(0, score), 'issues': issues, 'warnings': warnings}
    
    def _validate_numeric_claims(self, insight: Dict[str, Any]) -> Dict[str, Any]:
        """Validate plausibility of numeric claims."""
        issues = []
        score = 100
        
        # Extract numeric claims from text
        full_text = f"{insight.get('hook', '')} {insight.get('explanation', '')} {insight.get('action', '')}"
        
        # Find percentages
        percentages = re.findall(r'(\d+(?:\.\d+)?)\s*%', full_text)
        for pct in percentages:
            pct_val = float(pct)
            if pct_val > 100:
                issues.append(f"Implausible percentage: {pct_val}%")
                score -= 15
            if pct_val > 95:
                issues.append(f"Very high percentage (verify): {pct_val}%")
                score -= 5
        
        # Find risk multipliers (e.g., "3x higher", "5 times")
        multipliers = re.findall(r'(\d+(?:\.\d+)?)\s*(?:x|times)\s+(?:higher|greater|more|increased)', full_text.lower())
        for mult in multipliers:
            mult_val = float(mult)
            if mult_val > 20:
                issues.append(f"Implausibly high risk multiplier: {mult_val}x")
                score -= 15
        
        # Check for absolute risk numbers
        absolute_risks = re.findall(r'(\d+)\s+in\s+(\d+)', full_text)
        for numerator, denominator in absolute_risks:
            num, denom = int(numerator), int(denominator)
            if num > denom:
                issues.append(f"Invalid ratio: {num} in {denom}")
                score -= 20
        
        return {'score': max(0, score), 'issues': issues}
    
    def _validate_content_quality(self, insight: Dict[str, Any]) -> Dict[str, Any]:
        """Validate content quality and appropriateness."""
        issues = []
        warnings = []
        score = 100
        
        hook = insight.get('hook', '').lower()
        explanation = insight.get('explanation', '').lower()
        action = insight.get('action', '').lower()
        
        # Check if hook starts with "Did you know"
        if not hook.startswith('did you know'):
            warnings.append("Hook should start with 'Did you know'")
            score -= 5
        
        # Check for fear-mongering language
        fear_words = ['deadly', 'fatal', 'dangerous', 'terrifying', 'horrifying']
        for word in fear_words:
            if word in hook or word in explanation:
                warnings.append(f"Potentially fear-mongering language: '{word}'")
                score -= 5
        
        # Check for vague language
        vague_phrases = ['may', 'might', 'could possibly', 'perhaps', 'some studies suggest']
        vague_count = sum(1 for phrase in vague_phrases if phrase in explanation)
        if vague_count > 2:
            warnings.append("Too much hedging/vague language")
            score -= 10
        
        # Check for actionability
        action_verbs = ['speak', 'talk', 'consult', 'ask', 'schedule', 'start', 'try', 
                       'consider', 'visit', 'contact', 'discuss', 'check', 'get']
        has_action_verb = any(verb in action for verb in action_verbs)
        if not has_action_verb:
            issues.append("Action lacks clear action verb")
            score -= 15
        
        # Check for medical diagnosis/treatment claims
        diagnosis_words = ['diagnose', 'treat', 'cure', 'prescribe']
        for word in diagnosis_words:
            if word in action:
                issues.append(f"Action should not include medical diagnosis/treatment: '{word}'")
                score -= 20
        
        # Check cohort specificity
        cohort_params = insight.get('cohort_params', {})
        full_text = f"{hook} {explanation} {action}".lower()
        
        # Should mention cohort characteristics
        cohort_mentions = 0
        if 'age_group' in cohort_params:
            age = cohort_params['age_group']
            if age in full_text or 'age' in full_text:
                cohort_mentions += 1
        
        if 'gender' in cohort_params:
            gender = cohort_params['gender']
            if gender in full_text:
                cohort_mentions += 1
        
        if cohort_mentions == 0 and len(cohort_params) > 1:
            warnings.append("Insight doesn't reference specific cohort characteristics")
            score -= 10
        
        return {'score': max(0, score), 'issues': issues, 'warnings': warnings}
    
    def validate_batch(self, insights: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Validate a batch of insights and provide summary statistics.
        
        Args:
            insights: List of insights to validate
            
        Returns:
            Summary validation results
        """
        results = []
        
        for insight in insights:
            result = self.validate_insight(insight)
            results.append(result)
        
        # Summary statistics
        valid_count = sum(1 for r in results if r['overall_valid'])
        avg_score = sum(r['overall_score'] for r in results) / len(results) if results else 0
        
        # Count issues by type
        all_issues = []
        all_warnings = []
        for r in results:
            all_issues.extend(r['issues'])
            all_warnings.extend(r['warnings'])
        
        summary = {
            'total_insights': len(insights),
            'valid_insights': valid_count,
            'invalid_insights': len(insights) - valid_count,
            'average_score': round(avg_score, 2),
            'total_issues': len(all_issues),
            'total_warnings': len(all_warnings),
            'results': results
        }
        
        return summary
    
    def check_duplicates(self, insights: List[Dict[str, Any]], 
                        threshold: float = 0.85) -> List[Tuple[int, int, float]]:
        """
        Check for duplicate or highly similar insights using semantic similarity.
        
        Args:
            insights: List of insights
            threshold: Similarity threshold (0-1) above which insights are considered duplicates
            
        Returns:
            List of (index1, index2, similarity_score) tuples for duplicates
        """
        duplicates = []
        
        for i in range(len(insights)):
            for j in range(i + 1, len(insights)):
                # Compare full text
                text1 = f"{insights[i].get('hook', '')} {insights[i].get('explanation', '')}"
                text2 = f"{insights[j].get('hook', '')} {insights[j].get('explanation', '')}"
                
                similarity = SequenceMatcher(None, text1, text2).ratio()
                
                if similarity >= threshold:
                    duplicates.append((i, j, similarity))
        
        return duplicates